### Интеграция API

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from urllib import request, parse
import json

Загрузка тестового датасета

In [2]:
TEST_DATASET_PATH = '/Users/annasmejoukha/Desktop/ML_Business/project/datasets/test.csv'

In [3]:
df = pd.read_csv(TEST_DATASET_PATH, index_col=0)
df.head(2)

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25976 entries, 0 to 25975
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 25976 non-null  int64  
 1   Gender                             25976 non-null  object 
 2   Customer Type                      25976 non-null  object 
 3   Age                                25976 non-null  int64  
 4   Type of Travel                     25976 non-null  object 
 5   Class                              25976 non-null  object 
 6   Flight Distance                    25976 non-null  int64  
 7   Inflight wifi service              25976 non-null  int64  
 8   Departure/Arrival time convenient  25976 non-null  int64  
 9   Ease of Online booking             25976 non-null  int64  
 10  Gate location                      25976 non-null  int64  
 11  Food and drink                     25976 non-null  int

Разбиваем дата сет на X и y

In [5]:
X = df.drop('satisfaction', axis=1)
y = pd.DataFrame({'satisfaction': df['satisfaction']})

In [6]:
def get_prediction(data):
    """
    Данная функция позволяет получить предсказание от модели через API
    """
    myurl = "http://127.0.0.1:5000/predict"
    req = request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = data.to_json()
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [7]:
%%time
predictions = np.array(get_prediction(X))

CPU times: user 82.9 ms, sys: 15.2 ms, total: 98.1 ms
Wall time: 675 ms


In [8]:
y_true = np.where(y.satisfaction == 'satisfied', 1, 0)

In [9]:
preds = np.where(predictions == 'satisfied', 1, 0)
preds

array([1, 1, 0, ..., 0, 1, 0])

In [10]:
print(f'ROC AUC score: {roc_auc_score(y, preds)}')

ROC AUC score: 0.961491592533332
